# **Import Libraries**

In [3]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam

# **Define data directories**

In [4]:
train_data_dir = '/content/Training'
validation_data_dir = '/content/Validation'
img_width, img_height = 224, 224
batch_size = 32

# **Data Augmentation for training data**

In [5]:
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# **Data Augmentation for validation data (only rescaling)**

In [6]:
validation_datagen = ImageDataGenerator(rescale=1.0 / 255)

# **Create data generators**

In [7]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary'  # binary classification: 0 for male, 1 for female
)

validation_generator = validation_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary'
)

Found 47009 images belonging to 2 classes.
Found 11649 images belonging to 2 classes.


# **Load the VGG16 model with pre-trained weights**

In [8]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))

58889256/58889256 [==============================] - 0s 0us/step


# **Freeze the layers of the VGG16 model**

In [9]:
for layer in base_model.layers:
    layer.trainable = False

# **Create a custom top classifier**

In [10]:
model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# **Compile the model**

In [11]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

# **Train the model**

In [12]:
history = model.fit(train_generator, epochs=5, validation_data=validation_generator)

Epoch 1/5
1470/1470 [==============================] - 719s 480ms/step - loss: 0.3403 - accuracy: 0.8529 - val_loss: 0.2530 - val_accuracy: 0.8961
Epoch 2/5
1470/1470 [==============================] - 693s 471ms/step - loss: 0.2740 - accuracy: 0.8881 - val_loss: 0.1896 - val_accuracy: 0.9222
Epoch 3/5
1470/1470 [==============================] - 690s 469ms/step - loss: 0.2549 - accuracy: 0.8972 - val_loss: 0.1770 - val_accuracy: 0.9296
Epoch 4/5
1470/1470 [==============================] - 694s 472ms/step - loss: 0.2482 - accuracy: 0.9001 - val_loss: 0.1795 - val_accuracy: 0.9287
Epoch 5/5
1470/1470 [==============================] - 694s 472ms/step - loss: 0.2378 - accuracy: 0.9036 - val_loss: 0.2228 - val_accuracy: 0.9092


# **Save the trained model**

In [13]:
model.save('gender_classification_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


## **Evaluate the model**

In [14]:
evaluation = model.evaluate(validation_generator)
print(f"Validation Accuracy: {evaluation[1] * 100:.2f}%")

365/365 [==============================] - 46s 126ms/step - loss: 0.2228 - accuracy: 0.9092
Validation Accuracy: 90.92%
